In [2]:
#automatic result extraction

In [2]:
import os
import glob

# Get the Downloads folder path
downloads_folder = os.path.expanduser("~/Downloads")

# Get all CSV files in the Downloads folder
list_of_files = glob.glob(os.path.join(downloads_folder, "*.csv"))

if list_of_files:  # Ensure there are CSV files
    latest_file = max(list_of_files, key=os.path.getmtime)
    print("Latest downloaded file:", latest_file)

    # Read the latest CSV file into a DataFrame
    import pandas as pd
    df = pd.read_csv(latest_file,delimiter=';', skiprows=7)
    df.head()
else:
    print("No CSV files found in Downloads.")


Latest downloaded file: C:\Users\tasni/Downloads\Résultat_de_la_recherche_dans_Espacenet_20250505_1914.csv


data cleaning

In [3]:
import pandas as pd

#renaming columns
df.rename(columns={
    'Titre': 'Title',
    'Inventeurs': 'Inventors',
    'Demandeurs': 'Applicants',
    'Numéro de publication': 'Publication number',
    'Priorité la plus ancienne': 'Earliest priority',
    'CIB': 'IPC',
    'CPC': 'CPC',
    'Date de publication': 'Publication date',
    'Publication la plus ancienne': 'Earliest publication',
    'Numéro de famille': 'Family number'
}, inplace=True)
#cleaning the date columns
df[['first publication date','second publication date']] = df['Publication date'].str.split(' ' , n=1 , expand= True)
#df['second publication date'] = df['second publication date'].str.strip('\n')
df['second publication date'] = df['second publication date'].str.strip('\r')
df['second publication date'] = df['second publication date'].str.strip('\n')

#first filing country 
df[['first publication number', 'second publication number']] = df['Publication number'].str.split(' ' , n=1 , expand=True)
#df['second publication number'] = df['second publication number'].str.strip('\n')
df['second publication number'] = df['second publication number'].str.strip('\r')
df['second publication number'] = df['second publication number'].str.strip('\n')


df.rename(columns={'No': 'id'}, inplace=True)
if 'Unnamed: 11' in df.columns:
    df.drop(columns=['Unnamed: 11','Publication date'], inplace=True)

df['Family number'] = pd.to_numeric(df['Family number'], errors='coerce')
df.rename(columns={'Family number': 'family number'}, inplace=True)

In [4]:
df.head()

,id,Title,Inventors,Applicants,Publication number,Earliest priority,IPC,CPC,Earliest publication,family number,first publication date,second publication date,first publication number,second publication number
0,1,APPAREIL D'AUTOMATISATION À ANALYSE DE RÉSEAU ...,HILSCHER HANS-JÜRGEN [DE],HILSCHER GES FUER SYSTEMAUTOMATION MBH [DE] \r...,EP3648416A1 \r\nEP3648416B1,2018-11-05,H04L12/40 \r\nH04L12/413,H04L12/40032 (EP) \r\nH04L12/413 (EP) \r\nH04L...,2020-05-06,68470224,2020-05-06,2023-08-16,EP3648416A1,EP3648416B1
1,2,SYSTEM AND METHOD FOR SELECTION OF CLOUD SERVI...,RAMAMURTHY ARUN [IN] \r\nGHAROTE MANGESH SHARA...,TATA CONSULTANCY SERVICES LTD [IN],US11405415B2 \r\nUS2021176266A1,2019-12-06,H04L29/06 \r\nH04L29/08 \r\nG06F9/455 \r\nG06F...,"G06F9/45558 (US) \r\nG06F9/5072 (EP,US) \r\nG0...",2021-06-09,69941192,2021-06-10,2022-08-02,US11405415B2,US2021176266A1
2,3,SECURITY-ENHANCED CLOUD SYSTEM AND SECURITY MA...,LEE CHUNG JONG [KR],LEE CHUNG JONG [KR],US2014250500A1 \r\nUS9124579B2,2011-09-29,H04L29/06 \r\nG06F21/32 \r\nG06K9/00 \r\nH04L2...,G06F15/16 (KR) \r\nG06F21/30 (KR) \r\nG06F21/3...,2013-04-04,47996604,2014-09-04,2015-09-01,US2014250500A1,US9124579B2
3,4,"METHOD, APPARATUS, AND SYSTEM FOR PROTECTING C...",ZHANG JINGBIN [CN] \r\nHE CHENGDONG [CN],HUAWEI TECH CO LTD [CN],US2014126723A1 \r\nUS9203614B2,2011-11-09,H04L9/08 \r\nH04L29/06 \r\nH04L9/14,"H04L9/0822 (EP,US) \r\nH04L9/0825 (EP,US) \r\n...",2012-12-06,47258336,2014-05-08,2015-12-01,US2014126723A1,US9203614B2
4,5,TECHNIQUES FOR CLOUD SECURITY MONITORING AND T...,KIRTI GANESH [US] \r\nGUPTA ROHIT [US] \r\nBIS...,ORACLE INT CORP [US],US10958679B2 \r\nUS2017295199A1,2013-12-13,H04L29/06,"H04L63/1416 (EP,US) \r\nH04L63/1425 (US) \r\nH...",2015-06-18,53369924,2017-10-12,2021-03-23,US10958679B2,US2017295199A1


paralell processing family members

In [5]:
# Calculate the number of rows for each part
n = len(df) // 3

# Split the DataFrame into three parts
df1 = df.iloc[:n].copy()       # First part
df2 = df.iloc[n:2*n].copy()    # Second part
df3 = df.iloc[2*n:].copy()     # Third part

In [6]:
len(df1), len(df2), len(df3)

(166, 166, 168)

In [7]:
import concurrent.futures
import os
import requests
import time
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv

# Global token cache for multiple credentials
TOKENS = {}
TOKENS_EXPIRY = {}

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()
CONSUMER_KEY1 = os.getenv("CONSUMER_KEY_2").strip()
CONSUMER_SECRET1 = os.getenv("CONSUMER_SECRET_2").strip()

CREDENTIALS = [
    {"key": CONSUMER_KEY, "secret": CONSUMER_SECRET},
    {"key": CONSUMER_KEY1, "secret": CONSUMER_SECRET1}
]

def get_access_token(key_index: int = 0) -> str:
    if key_index not in TOKENS or time.time() >= TOKENS_EXPIRY[key_index]:
        creds = CREDENTIALS[key_index]
        data = {
            "grant_type": "client_credentials",
            "client_id": creds["key"],
            "client_secret": creds["secret"]
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
        response.raise_for_status()
        TOKENS[key_index] = response.json()["access_token"]
        TOKENS_EXPIRY[key_index] = time.time() + 3500
    return TOKENS[key_index]

def validate_patent_number(patent: str) -> bool:
    return bool(patent and len(patent.strip()) >= 4)

def extract_jurisdictions_and_members(data: dict) -> dict:
    jurisdictions = set()
    family_members = []
    world_data = data.get('ops:world-patent-data', {})
    patent_family = world_data.get('ops:patent-family', {})
    members = patent_family.get('ops:family-member', []) or []
    if isinstance(members, dict):
        members = [members]

    for member in members:
        pub_ref = member.get('publication-reference', {})
        docs = pub_ref.get('document-id', []) or []
        if isinstance(docs, dict):
            docs = [docs]
        for doc in docs:
            if doc.get('@document-id-type') == 'docdb':
                country = doc.get('country')
                country = country.get('$') if isinstance(country, dict) else country
                doc_number = doc.get('doc-number')
                doc_number = doc_number.get('$') if isinstance(doc_number, dict) else doc_number
                kind = doc.get('kind')
                kind = kind.get('$') if isinstance(kind, dict) else kind
                if country and doc_number and kind:
                    jurisdictions.add(country)
                    family_members.append(f"{country}{doc_number}{kind}")
    return {
        'jurisdictions': sorted(jurisdictions),
        'family_members': sorted(set(family_members))
    }

def process_patent(patent: str, key_index: int) -> dict:
    if not validate_patent_number(patent):
        return {'jurisdictions': None, 'family_members': None, 'error': 'Invalid patent number'}
    try:
        token = get_access_token(key_index)
        url = f"{BASE_URL}/family/publication/docdb/{quote(patent)}"
        headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code in (403, 404):
            return {'jurisdictions': None, 'family_members': None, 'error': f'API returned {response.status_code}'}
        response.raise_for_status()
        data = extract_jurisdictions_and_members(response.json())
        data['error'] = None
        return data
    except Exception as e:
        return {'jurisdictions': None, 'family_members': None, 'error': str(e)}

def process_dataframe_parallel(df: pd.DataFrame, patent_col: str, max_workers: int = 10) -> pd.DataFrame:
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    patents = df[patent_col].tolist()
    N = len(patents)
    mid = N // 2
    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_patent = {}
        for i, patent in enumerate(patents):
            key_index = 0 if i < mid else 1
            future = executor.submit(process_patent, patent, key_index)
            future_to_patent[future] = patent
        for future in concurrent.futures.as_completed(future_to_patent):
            patent = future_to_patent[future]
            result = future.result()
            results[patent] = result
            if result['error'] is not None:
                print(f"Error for patent {patent}: {result['error']}")
            time.sleep(0.1)

    df['family_jurisdictions'] = df[patent_col].map(lambda p: results[p]['jurisdictions'])
    df['family_members'] = df[patent_col].map(lambda p: results[p]['family_members'])
    return df

# Example usage
if __name__ == "__main__":
    # Create a sample DataFrame (fixing the original issue where df2 was used before definition)
    #df2 = pd.DataFrame({'first publication number': ['US1234567A', 'INVALID', 'EP1234567A']})
    df1 = process_dataframe_parallel(df2, 'first publication number', max_workers=4)
    print(df1)

Error for patent CN110569041A: API returned 403
Error for patent CN106603533A: API returned 403
Error for patent US9197419B1: API returned 403
Error for patent CN105100076A: API returned 403
Error for patent CN108134783A: API returned 403
Error for patent US10530815B2: API returned 403
Error for patent CN118250101A: API returned 403
Error for patent US10230813B2: API returned 403
Error for patent CN112615744A: API returned 403
Error for patent CN103618703A: API returned 403
Error for patent CN116506200A: API returned 403
Error for patent CN109639705A: API returned 403
Error for patent CN105897650A: API returned 403
Error for patent CN111600889A: API returned 403
Error for patent CN114448668A: API returned 403
Error for patent CN103746988A: API returned 403
Error for patent CN117955719A: API returned 403
Error for patent CN217279669U: API returned 403
      id                                              Title  \
166  167                       Cross Cloud Tenant Discovery   
167  168  P

In [8]:
df1['family_members'].isnull().sum(), df1['family_jurisdictions'].isnull().sum()

(np.int64(18), np.int64(18))

In [9]:
#3 threads
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import threading

class PatentsSearch:
    def __init__(self, headless=True):
        """Initialize the scraper with enhanced compatibility options."""
        options = uc.ChromeOptions()
        if headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--disable-extensions')
        
        try:
            self.driver = uc.Chrome(
                options=options,
                use_subprocess=True,
                version_main=None,
                suppress_welcome=True,
                debug=False
            )
            self.driver.set_page_load_timeout(30)
            self.driver.set_window_size(1920, 1080)
        except Exception as e:
            print(f"Failed to initialize ChromeDriver: {e}")
            print("Trying alternative initialization method...")
            self.driver = uc.Chrome(
                options=options,
                driver_executable_path=None
            )

    def add_random_delay(self, min_seconds=1, max_seconds=3):
        """Add a random delay to mimic human behavior."""
        time.sleep(random.uniform(min_seconds, max_seconds))

    def get_page_html(self, url):
        """Navigate to the given URL and return the page HTML."""
        try:
            print(f"Navigating to: {url}")
            self.driver.get(url)
            WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "h5"))
            )
            self.add_random_delay(3, 5)
            return self.driver.page_source
        except TimeoutException:
            print("Timed out waiting for the page to load.")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def parse_html(self, html):
        """Parse the HTML and extract all span elements inside the 'Published as' content."""
        soup = BeautifulSoup(html, 'html.parser')
        published_as_element = soup.find(lambda tag: tag.name == "h5" and ("Publié en tant que" in tag.text or "Published as" in tag.text))
        if published_as_element:
            content_element = published_as_element.find_next_sibling("span")
            if content_element:
                spans = content_element.find_all('span')
                return [span.get_text(strip=True) for span in spans]
        return []

    def close(self):
        """Close the browser when done."""
        if self.driver:
            self.driver.quit()

def process_rows(df, indices):
    """Process a subset of DataFrame rows using a dedicated PatentsSearch instance."""
    scraper = PatentsSearch(headless=False)  # Set to False as per task requirement for visible windows
    try:
        for index in indices:
            row = df.loc[index]
            url = f"https://worldwide.espacenet.com/patent/search/family/{row['family number']}/publication/{row['first publication number']}?q=hydrogen%20battery"
            html = scraper.get_page_html(url)
            if html:
                family_members = scraper.parse_html(html)
                df.at[index, 'family_members'] = family_members
            else:
                print(f"Failed to retrieve the page HTML for {row['first publication number']}.")
    finally:
        scraper.close()

if __name__ == "__main__":
    # Assuming df is defined elsewhere with 'family number' and 'first publication number' columns
    
    df2['family_members'] = None

    # Split the DataFrame indices into three parts
    indices = df2.index.tolist()
    n = len(indices)
    part_size = n // 3
    remainder = n % 3
    parts = []
    start = 0
    for i in range(3):
        if i < remainder:
            end = start + part_size + 1
        else:
            end = start + part_size
        parts.append(indices[start:end])
        start = end

    # Create three threads, each with its own PatentsSearch instance
    threads = []
    for part in parts:
        thread = threading.Thread(target=process_rows, args=(df2, part))
        threads.append(thread)

    # Start all three threads to run three browser windows concurrently
    for thread in threads:
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    print("All threads finished.")
    # df now contains the 'family_members' column filled from all three threads

Navigating to: https://worldwide.espacenet.com/patent/search/family/56631046/publication/CN105871876A?q=hydrogen%20battery
Failed to initialize ChromeDriver: [WinError 183] Impossible de créer un fichier déjà existant: 'C:\\Users\\tasni\\appdata\\roaming\\undetected_chromedriver\\undetected\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\tasni\\appdata\\roaming\\undetected_chromedriver\\undetected_chromedriver.exe'
Trying alternative initialization method...
Navigating to: https://worldwide.espacenet.com/patent/search/family/61393579/publication/CN107786495A?q=hydrogen%20battery
Failed to initialize ChromeDriver: [WinError 183] Impossible de créer un fichier déjà existant: 'C:\\Users\\tasni\\appdata\\roaming\\undetected_chromedriver\\undetected\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\tasni\\appdata\\roaming\\undetected_chromedriver\\undetected_chromedriver.exe'
Trying alternative initialization method...
Navigating to: https://worldwide.espacenet.com/patent/search/f

In [10]:
df2['family_members'].isnull().sum()  # Check for any missing values in the 'family_members' column
# Check for any empty lists in the 'family_members' column

np.int64(23)

In [16]:
df2['family_jurisdictions'].isnull().sum()  # Check for any missing values in the 'family_jurisdictions' column

np.int64(18)

In [11]:
(df2['family_members'].apply(lambda x: isinstance(x, list) and len(x) == 0)).sum()  

np.int64(0)

In [12]:
import concurrent.futures
import os
import requests
import time
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv

# Global token cache for multiple credentials
TOKENS = {}
TOKENS_EXPIRY = {}

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()
CONSUMER_KEY1 = os.getenv("CONSUMER_KEY_1").strip()
CONSUMER_SECRET1 = os.getenv("CONSUMER_SECRET_1").strip()

CREDENTIALS = [
    {"key": CONSUMER_KEY, "secret": CONSUMER_SECRET},
    {"key": CONSUMER_KEY1, "secret": CONSUMER_SECRET1}
]

def get_access_token(key_index: int = 0) -> str:
    if key_index not in TOKENS or time.time() >= TOKENS_EXPIRY[key_index]:
        creds = CREDENTIALS[key_index]
        data = {
            "grant_type": "client_credentials",
            "client_id": creds["key"],
            "client_secret": creds["secret"]
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
        response.raise_for_status()
        TOKENS[key_index] = response.json()["access_token"]
        TOKENS_EXPIRY[key_index] = time.time() + 3500
    return TOKENS[key_index]

def validate_patent_number(patent: str) -> bool:
    return bool(patent and len(patent.strip()) >= 4)

def extract_jurisdictions_and_members(data: dict) -> dict:
    jurisdictions = set()
    family_members = []
    world_data = data.get('ops:world-patent-data', {})
    patent_family = world_data.get('ops:patent-family', {})
    members = patent_family.get('ops:family-member', []) or []
    if isinstance(members, dict):
        members = [members]

    for member in members:
        pub_ref = member.get('publication-reference', {})
        docs = pub_ref.get('document-id', []) or []
        if isinstance(docs, dict):
            docs = [docs]
        for doc in docs:
            if doc.get('@document-id-type') == 'docdb':
                country = doc.get('country')
                country = country.get('$') if isinstance(country, dict) else country
                doc_number = doc.get('doc-number')
                doc_number = doc_number.get('$') if isinstance(doc_number, dict) else doc_number
                kind = doc.get('kind')
                kind = kind.get('$') if isinstance(kind, dict) else kind
                if country and doc_number and kind:
                    jurisdictions.add(country)
                    family_members.append(f"{country}{doc_number}{kind}")
    return {
        'jurisdictions': sorted(jurisdictions),
        'family_members': sorted(set(family_members))
    }

def process_patent(patent: str, key_index: int) -> dict:
    if not validate_patent_number(patent):
        return {'jurisdictions': None, 'family_members': None, 'error': 'Invalid patent number'}
    try:
        token = get_access_token(key_index)
        url = f"{BASE_URL}/family/publication/docdb/{quote(patent)}"
        headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code in (403, 404):
            return {'jurisdictions': None, 'family_members': None, 'error': f'API returned {response.status_code}'}
        response.raise_for_status()
        data = extract_jurisdictions_and_members(response.json())
        data['error'] = None
        return data
    except Exception as e:
        return {'jurisdictions': None, 'family_members': None, 'error': str(e)}

def process_dataframe_parallel(df: pd.DataFrame, patent_col: str, max_workers: int = 10) -> pd.DataFrame:
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    patents = df[patent_col].tolist()
    N = len(patents)
    mid = N // 2
    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_patent = {}
        for i, patent in enumerate(patents):
            key_index = 0 if i < mid else 1
            future = executor.submit(process_patent, patent, key_index)
            future_to_patent[future] = patent
        for future in concurrent.futures.as_completed(future_to_patent):
            patent = future_to_patent[future]
            result = future.result()
            results[patent] = result
            if result['error'] is not None:
                print(f"Error for patent {patent}: {result['error']}")
            time.sleep(0.1)

    df['family_jurisdictions'] = df[patent_col].map(lambda p: results[p]['jurisdictions'])
    df['family_members'] = df[patent_col].map(lambda p: results[p]['family_members'])
    return df

# Example usage
if __name__ == "__main__":
    # Create a sample DataFrame (fixing the original issue where df2 was used before definition)
    #df2 = pd.DataFrame({'first publication number': ['US1234567A', 'INVALID', 'EP1234567A']})
    df3 = process_dataframe_parallel(df3, 'first publication number', max_workers=4)
    print(df3)

Error for patent CN105487936A: API returned 403
Error for patent CN109995702A: API returned 403
Error for patent CN103812862A: API returned 403
Error for patent CN115622790A: API returned 403
Error for patent CN106856467A: API returned 403
Error for patent CN111131599A: API returned 403
Error for patent CN105227581A: API returned 403
Error for patent US2015188939A1: API returned 403
Error for patent CN112769782A: API returned 403
Error for patent US11616790B2: API returned 403
Error for patent CN113395271A: API returned 403
Error for patent CN104125251A: API returned 403
Error for patent US2023353462A1: API returned 403
Error for patent US11159577B2: API returned 403
Error for patent CN103914897A: API returned 403
Error for patent CN116226865A: API returned 403
Error for patent CN109561074A: API returned 403
Error for patent US2015033285A1: API returned 403
Error for patent CN111431914A: API returned 403
Error for patent CN111526156A: API returned 403
Error for patent CN103258160A: API

In [13]:
df3['family_jurisdictions'].isnull().sum()
  # Check for any missing values in the 'family_jurisdictions' column


np.int64(103)

In [14]:
(df3['family_jurisdictions'].apply(lambda x: isinstance(x, list) and len(x) == 0)).sum()

np.int64(0)

In [15]:
df3['family_jurisdictions']

332    None
333    None
334    None
335    None
336    None
       ... 
495    None
496    None
497    None
498    [KR]
499    None
Name: family_jurisdictions, Length: 168, dtype: object

In [ ]:
import concurrent.futures
import os
import requests
import time
import logging
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv
import random
from functools import lru_cache
import threading

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Global token cache
TOKEN = None
TOKEN_EXPIRY = 0

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Rate limiting settings - much more conservative
REQUEST_INTERVAL = 1.0  # 1 second between requests
MAX_WORKERS = 3         # Limit concurrent connections
MAX_RETRIES = 5         # More retries
TIMEOUT = 30            # Longer timeout

# Rate limiter with token bucket algorithm
class RateLimiter:
    def __init__(self, rate=1, capacity=5):
        self.rate = rate            # tokens per second
        self.capacity = capacity    # maximum tokens
        self.tokens = capacity      # current tokens
        self.last_refill = time.time()
        self.lock = threading.Lock()
    
    def consume(self, tokens=1):
        with self.lock:
            self._refill()
            if self.tokens >= tokens:
                self.tokens -= tokens
                return True
            else:
                sleep_time = (tokens - self.tokens) / self.rate
                logger.debug(f"Rate limited, sleeping for {sleep_time:.2f}s")
                time.sleep(sleep_time)
                self._refill()
                self.tokens -= tokens
                return True
    
    def _refill(self):
        now = time.time()
        elapsed = now - self.last_refill
        new_tokens = elapsed * self.rate
        self.tokens = min(self.capacity, self.tokens + new_tokens)
        self.last_refill = now

# Create a rate limiter - 1 request per second, max 3 burst
rate_limiter = RateLimiter(rate=0.5, capacity=3)

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()

# Token lock to prevent multiple simultaneous token requests
token_lock = threading.Lock()

def get_access_token(force_refresh=False) -> str:
    global TOKEN, TOKEN_EXPIRY
    
    with token_lock:
        if not force_refresh and TOKEN and time.time() < TOKEN_EXPIRY:
            return TOKEN
        
        logger.info("Getting new access token")
        data = {
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        
        for attempt in range(MAX_RETRIES):
            try:
                response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=TIMEOUT)
                response.raise_for_status()
                TOKEN = response.json()["access_token"]
                TOKEN_EXPIRY = time.time() + 3500  # Slightly less than 1 hour
                logger.info("New token acquired")
                return TOKEN
            except Exception as e:
                wait_time = (2 ** attempt) + random.uniform(0, 1)
                logger.warning(f"Failed to get token (attempt {attempt+1}/{MAX_RETRIES}): {str(e)}. Retrying in {wait_time:.2f}s")
                time.sleep(wait_time)
        
        logger.error("Failed to get access token after multiple attempts")
        raise Exception("Failed to get access token")

def validate_patent_number(patent: str) -> bool:
    return bool(patent and isinstance(patent, str) and len(patent.strip()) >= 4)

@lru_cache(maxsize=1000)
def process_patent_with_cache(patent: str) -> dict:
    """Cached version of process_patent to avoid redundant API calls"""
    return process_patent_internal(patent)

def backoff_sleep(attempt, base=2.0, max_sleep=60):
    """Calculate and sleep for exponential backoff time with jitter"""
    sleep_time = min(base ** attempt + random.uniform(0, 1), max_sleep)
    logger.debug(f"Backing off for {sleep_time:.2f}s")
    time.sleep(sleep_time)

def process_patent_internal(patent: str) -> dict:
    """Internal function that actually processes the patent"""
    if not validate_patent_number(patent):
        logger.warning(f"Invalid patent number: '{patent}'")
        return {'jurisdictions': None, 'family_members': None}
    
    for attempt in range(MAX_RETRIES):
        try:
            # Consume a token from the rate limiter
            rate_limiter.consume()
            
            # Get a fresh token if needed
            token = get_access_token()
            
            url = f"{BASE_URL}/family/publication/docdb/{quote(patent)}"
            headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
            
            logger.debug(f"Requesting data for patent: {patent}")
            response = requests.get(url, headers=headers, timeout=TIMEOUT)
            
            if response.status_code == 401:  # Unauthorized - token expired
                logger.warning("Token expired, getting a new one")
                get_access_token(force_refresh=True)
                backoff_sleep(attempt)
                continue
                
            if response.status_code == 403:
                logger.warning(f"Access forbidden for patent {patent}. Possibly rate limited.")
                backoff_sleep(attempt + 2)  # More aggressive backoff for rate limiting
                continue
                
            if response.status_code == 404:
                logger.info(f"No data found for patent {patent}")
                return {'jurisdictions': None, 'family_members': None}
                
            response.raise_for_status()
            result = extract_jurisdictions_and_members(response.json())
            logger.debug(f"Successfully processed patent {patent}")
            return result
            
        except requests.exceptions.Timeout:
            logger.warning(f"Timeout for patent {patent} (attempt {attempt+1}/{MAX_RETRIES})")
            backoff_sleep(attempt)
            continue
            
        except requests.exceptions.ConnectionError as e:
            logger.warning(f"Connection error for patent {patent} (attempt {attempt+1}/{MAX_RETRIES}): {str(e)}")
            backoff_sleep(attempt + 1)  # More aggressive backoff for connection errors
            continue
            
        except Exception as e:
            logger.warning(f"Error processing patent {patent} (attempt {attempt+1}/{MAX_RETRIES}): {str(e)}")
            backoff_sleep(attempt)
            continue
    
    logger.error(f"Failed to process patent {patent} after {MAX_RETRIES} attempts")
    return {'jurisdictions': None, 'family_members': None}

def extract_jurisdictions_and_members(data: dict) -> dict:
    try:
        jurisdictions = set()
        family_members = []
        world_data = data.get('ops:world-patent-data', {})
        patent_family = world_data.get('ops:patent-family', {})
        members = patent_family.get('ops:family-member', []) or []
        if isinstance(members, dict):
            members = [members]

        for member in members:
            pub_ref = member.get('publication-reference', {})
            docs = pub_ref.get('document-id', []) or []
            if isinstance(docs, dict):
                docs = [docs]
            for doc in docs:
                if doc.get('@document-id-type') == 'docdb':
                    country = doc.get('country')
                    country = country.get('$') if isinstance(country, dict) else country
                    doc_number = doc.get('doc-number')
                    doc_number = doc_number.get('$') if isinstance(doc_number, dict) else doc_number
                    kind = doc.get('kind')
                    kind = kind.get('$') if isinstance(kind, dict) else kind
                    if country and doc_number and kind:
                        jurisdictions.add(country)
                        family_members.append(f"{country}{doc_number}{kind}")
        return {
            'jurisdictions': sorted(jurisdictions),
            'family_members': sorted(set(family_members))
        }
    except Exception as e:
        logger.error(f"Error extracting jurisdictions and members: {str(e)}")
        return {'jurisdictions': None, 'family_members': None}

def process_patent(patent: str) -> dict:
    """Wrapper function to call the cached version"""
    # Skip None/NaN values
    if pd.isna(patent) or patent is None:
        return {'jurisdictions': None, 'family_members': None}
    return process_patent_with_cache(patent)

def process_dataframe_parallel(df: pd.DataFrame, patent_col: str, chunk_size=50) -> pd.DataFrame:
    """Process patents in smaller chunks to avoid overwhelming the API"""
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    
    # Make a copy to avoid modifying the original
    result_df = df.copy()
    patents = df[patent_col].tolist()
    total_patents = len(patents)
    results = {}
    
    logger.info(f"Starting to process {total_patents} patents in chunks of {chunk_size}")
    
    # Get an access token before we start processing
    get_access_token()
    
    # Process in chunks to avoid overwhelming the API
    for chunk_start in range(0, total_patents, chunk_size):
        chunk_end = min(chunk_start + chunk_size, total_patents)
        current_chunk = patents[chunk_start:chunk_end]
        
        logger.info(f"Processing chunk {chunk_start//chunk_size + 1}/{(total_patents+chunk_size-1)//chunk_size} " +
                   f"(patents {chunk_start+1}-{chunk_end})")
        
        # Use ThreadPoolExecutor for I/O-bound API calls
        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            # Submit all tasks for this chunk
            future_to_patent = {executor.submit(process_patent, p): p for p in current_chunk}
            
            for future in concurrent.futures.as_completed(future_to_patent):
                patent = future_to_patent[future]
                try:
                    results[patent] = future.result()
                except Exception as e:
                    logger.error(f"Unhandled exception for patent {patent}: {str(e)}")
                    results[patent] = {'jurisdictions': None, 'family_members': None}
        
        # Calculate success rate for this chunk
        chunk_success = sum(1 for p in current_chunk if results.get(p, {}).get('jurisdictions') is not None)
        logger.info(f"Chunk complete: {chunk_success}/{len(current_chunk)} successful " +
                   f"({chunk_success/len(current_chunk)*100:.1f}%)")
        
        # If success rate is too low, pause to avoid rate limiting
        if chunk_success < len(current_chunk) * 0.5:
            pause_time = 60  # 1 minute pause
            logger.warning(f"Low success rate detected, pausing for {pause_time} seconds")
            time.sleep(pause_time)
    
    logger.info(f"Completed processing {total_patents} patents")
    
    # Apply results to the dataframe
    result_df['family_jurisdictions'] = result_df[patent_col].map(lambda p: results.get(p, {}).get('jurisdictions'))
    result_df['family_members'] = result_df[patent_col].map(lambda p: results.get(p, {}).get('family_members'))
    
    # Log statistics
    success_count = sum(1 for r in results.values() if r['jurisdictions'] is not None)
    logger.info(f"Successfully processed {success_count} out of {total_patents} patents ({success_count/total_patents*100:.1f}%)")
    
    return result_df

# Example usage:
if __name__ == "__main__":
    # df = pd.read_csv('your_patents.csv')
    df3 = process_dataframe_parallel(df2, 'first publication number', chunk_size=50)
    #processed_df[['first publication number', 'family_jurisdictions', 'family_members']]

2025-05-05 17:21:31,416 - INFO - Starting to process 220 patents in chunks of 50
2025-05-05 17:21:31,416 - INFO - Getting new access token
2025-05-05 17:21:31,702 - INFO - New token acquired
2025-05-05 17:21:31,703 - INFO - Processing chunk 1/5 (patents 1-50)
2025-05-05 17:21:33,973 - WARNING - Access forbidden for patent US11794779B2. Possibly rate limited.
2025-05-05 17:21:39,964 - WARNING - Access forbidden for patent US10438486B2. Possibly rate limited.
2025-05-05 17:21:41,933 - WARNING - Access forbidden for patent US11687079B2. Possibly rate limited.
2025-05-05 17:21:43,978 - WARNING - Access forbidden for patent US11794779B2. Possibly rate limited.
2025-05-05 17:21:46,081 - WARNING - Access forbidden for patent US10438486B2. Possibly rate limited.
2025-05-05 17:21:48,006 - WARNING - Access forbidden for patent US11687079B2. Possibly rate limited.
2025-05-05 17:21:52,580 - WARNING - Access forbidden for patent US11794779B2. Possibly rate limited.
2025-05-05 17:21:55,175 - WARNING

In [ ]:
import concurrent.futures
import os
import requests
import time
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv

# Global token cache
TOKEN = None
TOKEN_EXPIRY = 0

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()

def get_access_token() -> str:
    global TOKEN, TOKEN_EXPIRY
    if TOKEN and time.time() < TOKEN_EXPIRY:
        return TOKEN
    data = {
        "grant_type": "client_credentials",
        "client_id": CONSUMER_KEY,
        "client_secret": CONSUMER_SECRET
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
    response.raise_for_status()
    TOKEN = response.json()["access_token"]
    TOKEN_EXPIRY = time.time() + 3500
    return TOKEN

def validate_patent_number(patent: str) -> bool:
    return bool(patent and len(patent.strip()) >= 4)

def extract_jurisdictions_and_members(data: dict) -> dict:
    jurisdictions = set()
    family_members = []
    world_data = data.get('ops:world-patent-data', {})
    patent_family = world_data.get('ops:patent-family', {})
    members = patent_family.get('ops:family-member', []) or []
    if isinstance(members, dict):
        members = [members]

    for member in members:
        pub_ref = member.get('publication-reference', {})
        docs = pub_ref.get('document-id', []) or []
        if isinstance(docs, dict):
            docs = [docs]
        for doc in docs:
            if doc.get('@document-id-type') == 'docdb':
                country = doc.get('country')
                country = country.get('$') if isinstance(country, dict) else country
                doc_number = doc.get('doc-number')
                doc_number = doc_number.get('$') if isinstance(doc_number, dict) else doc_number
                kind = doc.get('kind')
                kind = kind.get('$') if isinstance(kind, dict) else kind
                if country and doc_number and kind:
                    jurisdictions.add(country)
                    family_members.append(f"{country}{doc_number}{kind}")
    return {
        'jurisdictions': sorted(jurisdictions),
        'family_members': sorted(set(family_members))
    }

def process_patent(patent: str) -> dict:
    if not validate_patent_number(patent):
        return {'jurisdictions': None, 'family_members': None, 'error': 'Invalid patent number'}
    try:
        token = get_access_token()
        url = f"{BASE_URL}/family/publication/docdb/{quote(patent)}"
        headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code in (403, 404):
            return {'jurisdictions': None, 'family_members': None, 'error': f'API returned {response.status_code}'}
        response.raise_for_status()
        data = extract_jurisdictions_and_members(response.json())
        data['error'] = None
        return data
    except Exception as e:
        return {'jurisdictions': None, 'family_members': None, 'error': str(e)}

def process_dataframe_parallel(df: pd.DataFrame, patent_col: str, max_workers: int = 10) -> pd.DataFrame:
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    patents = df[patent_col].tolist()
    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_patent = {executor.submit(process_patent, p): p for p in patents}
        for future in concurrent.futures.as_completed(future_to_patent):
            patent = future_to_patent[future]
            result = future.result()
            results[patent] = result
            if result['error'] is not None:
                print(f"Error for patent {patent}: {result['error']}")
            time.sleep(0.1)

    df['family_jurisdictions'] = df[patent_col].map(lambda p: results[p]['jurisdictions'])
    df['family_members'] = df[patent_col].map(lambda p: results[p]['family_members'])
    return df

# Example usage:
if __name__ == "__main__":
    # Example DataFrame (uncomment and adjust as needed)
    # df = pd.read_csv('your_patents.csv')
    # For demonstration, create a sample DataFrame
    #df2 = pd.DataFrame({'first publication number': ['US1234567A', 'INVALID', 'EP1234567A']})
    df2 = process_dataframe_parallel(df2, 'first publication number', max_workers=10)
    print(df2)

Error for patent US10438486B2: API returned 403
Error for patent US10782699B2: API returned 403
Error for patent US11711906B2: API returned 403
Error for patent WO2023173678A1: API returned 403
Error for patent EP3799752A1: API returned 403
Error for patent KR102658924B1: API returned 403
Error for patent US12099351B2: API returned 403
Error for patent US11144067B2: API returned 403
Error for patent US11794779B2: API returned 403
Error for patent US11687079B2: API returned 403
Error for patent US2022309925A1: API returned 403
Error for patent US11158021B2: API returned 403
Error for patent US11079771B2: API returned 403
Error for patent CN104076766A: API returned 403
Error for patent KR102636871B1: API returned 403
Error for patent US10545507B2: API returned 403
Error for patent US2019066409A1: API returned 403
Error for patent US11724677B2: API returned 403
Error for patent US11669653B2: API returned 403
Error for patent US11263787B2: API returned 403
Error for patent CN109410619A: AP

In [ ]:
df2[['first publication number', 'family_jurisdictions', 'family_members']].isnull().sum()  # Check for any missing values in the new columns

first publication number      0
family_jurisdictions        220
family_members              220
dtype: int64

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
import requests
import time
import threading
from urllib.parse import quote
import logging
import os
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Global variables for token management
TOKEN = None
TOKEN_EXPIRY = 0
TOKEN_LOCK = threading.Lock()

# Load environment variables
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

def to_dotted_format(patent: str) -> str:
    """Convert patent number to dotted format (e.g., US10438486B2 -> US.10438486.B2)."""
    country = patent[:2]
    number = patent[2:-2]
    kind = patent[-2:]
    return f"{country}.{number}.{kind}"

def validate_patent_number(patent: str) -> bool:
    """Basic validation for patent number format."""
    # Add your validation logic here (e.g., regex or length checks)
    return bool(patent and len(patent) > 5)  # Simplified example

def get_access_token() -> str:
    """Retrieve or refresh the access token in a thread-safe manner."""
    global TOKEN, TOKEN_EXPIRY
    with TOKEN_LOCK:
        if TOKEN and time.time() < TOKEN_EXPIRY:
            return TOKEN
        data = {
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        try:
            response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
            response.raise_for_status()
            TOKEN = response.json()["access_token"]
            TOKEN_EXPIRY = time.time() + 3500  # Token expires in ~58 minutes
            logging.info("Access token refreshed successfully")
            return TOKEN
        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to refresh token: {e}")
            raise

def extract_jurisdictions_and_members(data: dict) -> dict:
    """Extract jurisdictions and family members from API response."""
    # Placeholder for your extraction logic
    return {
        'jurisdictions': data.get('jurisdictions', []),
        'family_members': data.get('family_members', [])
    }

def process_patent(patent: str) -> dict:
    """Process a single patent number and handle API requests with retries."""
    logging.info(f"Processing patent {patent}")
    if not validate_patent_number(patent):
        logging.warning(f"Invalid patent number: {patent}")
        return {'jurisdictions': None, 'family_members': None, 'error': 'Invalid patent number'}

    dotted_patent = to_dotted_format(patent)
    max_retries = 3
    retry_delay = 1  # seconds

    for attempt in range(max_retries):
        try:
            token = get_access_token()
            url = f"{BASE_URL}/family/publication/docdb/{quote(dotted_patent)}"
            headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
            response = requests.get(url, headers=headers, timeout=15)
            
            if response.status_code == 403:
                logging.warning(f"Rate limit hit for {patent}, attempt {attempt + 1}/{max_retries}")
                if attempt < max_retries - 1:
                    time.sleep(retry_delay)
                    continue
                return {'jurisdictions': None, 'family_members': None, 'error': 'API returned 403 after retries'}
            elif response.status_code == 404:
                logging.warning(f"Patent not found: {patent}")
                return {'jurisdictions': None, 'family_members': None, 'error': 'Patent not found (404)'}
            
            response.raise_for_status()
            data = extract_jurisdictions_and_members(response.json())
            data['error'] = None
            logging.info(f"Successfully processed {patent}")
            return data
        except requests.exceptions.RequestException as e:
            logging.error(f"Request failed for {patent}, attempt {attempt + 1}/{max_retries}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                continue
            return {'jurisdictions': None, 'family_members': None, 'error': str(e)}

# Example usage
if __name__ == "__main__":
    from concurrent.futures import ThreadPoolExecutor
    #patents = ["US10438486B2", "EP1234567A1"]  # Replace with your list
    patents = df2['first publication number'].dropna().unique().tolist()  # Get unique patent numbers from DataFrame
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(process_patent, patents))
    for patent, result in zip(patents, results):
        print(f'{patent} : {result}')

2025-05-05 16:55:42,840 - INFO - Processing patent US11711906B2
2025-05-05 16:55:42,842 - INFO - Processing patent EP3799752A1
2025-05-05 16:55:42,843 - INFO - Processing patent US12099351B2
2025-05-05 16:55:42,843 - INFO - Processing patent US11794779B2
2025-05-05 16:55:42,846 - INFO - Processing patent WO2023173678A1
2025-05-05 16:55:43,141 - INFO - Access token refreshed successfully
2025-05-05 16:55:43,459 - INFO - Successfully processed US11794779B2
2025-05-05 16:55:43,460 - INFO - Processing patent US10782699B2
2025-05-05 16:55:43,462 - INFO - Successfully processed US11711906B2
2025-05-05 16:55:43,463 - INFO - Processing patent US10438486B2
2025-05-05 16:55:43,483 - INFO - Successfully processed US12099351B2
2025-05-05 16:55:43,484 - INFO - Processing patent US11687079B2
2025-05-05 16:55:43,492 - INFO - Successfully processed EP3799752A1
2025-05-05 16:55:43,493 - INFO - Processing patent US11144067B2
2025-05-05 16:55:43,536 - INFO - Successfully processed WO2023173678A1
2025-05-

US11711906B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
EP3799752A1 : {'jurisdictions': [], 'family_members': [], 'error': None}
US12099351B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
US11794779B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
WO2023173678A1 : {'jurisdictions': [], 'family_members': [], 'error': None}
US10782699B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
US10438486B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
US11687079B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
US11144067B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
KR102658924B1 : {'jurisdictions': [], 'family_members': [], 'error': None}
US10545507B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
KR102636871B1 : {'jurisdictions': [], 'family_members': [], 'error': None}
US11724677B2 : {'jurisdictions': [], 'family_members': [], 'error': None}
CN104076766A : {'jurisdictions': No

In [ ]:
df2['family_jurisdictions'] = [result['jurisdictions'] for result in results]

In [ ]:
(df2['family_jurisdictions'].apply(lambda x: isinstance(x, list) and len(x) == 0)).sum()


np.int64(166)

changing the pattern every few requests 

In [ ]:
# Improved multi-threaded scraper with rotating user‐agents, delays, and driver restarts every 25 requests
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import threading

# List of real-world User‑Agents to rotate
USER_AGENTS = [
    # Chrome on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    # Firefox on Mac
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0",
    # Safari on iPhone
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15A372 Safari/604.1",
    # Edge on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.43",
]

class PatentsSearch:
    def __init__(self, headless=True, user_agent=None):
        options = uc.ChromeOptions()
        if headless:
            options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument("--disable-extensions")
        # rotate user agent
        if user_agent:
            options.add_argument(f"--user-agent={user_agent}")
        else:
            options.add_argument(f"--user-agent={random.choice(USER_AGENTS)}")
        # random window size
        width = random.randint(1000, 1920)
        height = random.randint(700, 1080)
        options.add_argument(f"--window-size={width},{height}")

        # start driver
        self.driver = uc.Chrome(options=options, use_subprocess=True, suppress_welcome=True)
        self.driver.set_page_load_timeout(30)

    def add_random_delay(self, min_seconds=2, max_seconds=6):
        time.sleep(random.uniform(min_seconds, max_seconds))

    def get_page_html(self, url):
        try:
            self.driver.get(url)
            WebDriverWait(self.driver, random.randint(10,20)).until(
                EC.presence_of_element_located((By.TAG_NAME, "h5"))
            )
            self.add_random_delay()
            return self.driver.page_source
        except TimeoutException:
            print("Timeout loading page, retrying...")
            return None
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            return None

    def parse_html(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        published = soup.find(lambda tag: tag.name == "h5" and ("Publié en tant que" in tag.text or "Published as" in tag.text))
        if not published:
            return []
        span = published.find_next_sibling("span")
        return [s.get_text(strip=True) for s in span.find_all('span')] if span else []

    def close(self):
        self.driver.quit()


def process_rows(df, indices, batch_size=25):
    """
    Scrape in batches, restart driver and rotate UA every batch_size requests.
    """
    count = 0
    scraper = None

    def restart_scraper():
        nonlocal scraper
        if scraper:
            scraper.close()
        # choose new UA for each batch
        ua = random.choice(USER_AGENTS)
        scraper = PatentsSearch(headless=False, user_agent=ua)
        print(f"[Batch] New scraper with UA: {ua}")

    restart_scraper()

    for idx in indices:
        if count and count % batch_size == 0:
            restart_scraper()
        row = df.loc[idx]
        url = (f"https://worldwide.espacenet.com/patent/search/family/{row['family number']}" \
               f"/publication/{row['first publication number']}?q=hydrogen%20battery")
        html = scraper.get_page_html(url)
        if html:
            df.at[idx, 'family_members'] = scraper.parse_html(html)
        else:
            print(f"Failed at idx {idx}")
        count += 1
        # small random sleep between requests
        time.sleep(random.uniform(1, 3))

    if scraper:
        scraper.close()


if __name__ == "__main__":
    # Example: assume df2 already loaded
    df2['family_members'] = None
    indices = df2.index.tolist()
    # Use 3 threads splitting indices
    parts = [indices[i::3] for i in range(3)]
    threads = []
    for part in parts:
        t = threading.Thread(target=process_rows, args=(df2, part))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print("Done scraping.")

    # inspection
    empty = df2['family_members'].apply(lambda x: not x or pd.isna(x)).sum()
    print(f"Empty entries: {empty}/{len(df2)}")
